In [2]:
from create_dataset import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score


In [3]:
# image data path
DATAPATH = r'american_bankruptcy.csv'

logging.basicConfig(level=logging.INFO)
df_raw = load_data(DATAPATH)
dataset = create_dataset(df_raw)

INFO:root:Preprocessing data...
100%|██████████| 8971/8971 [01:08<00:00, 131.02it/s]
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/threadpoolctl.py:1010: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/Users/xinruoli/Desktop/Interpretation-DNN-main/create_dataset.py:76: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ra

In [4]:
ratios_df = ratios_dataframe(df_raw)
#ratios_df['test'] = df_raw["X9"] / df_raw["X16"]
#print(ratios_df['test'].describe())
for colunm in ratios_df.columns:
    if ratios_df[colunm].isnull().any():
        print(ratios_df[colunm].isnull())
# print(ratios_df['X9/X16'].head())

/Users/xinruoli/Desktop/Interpretation-DNN-main/create_dataset.py:76: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ratios_df[column] = df["X{}".format(i+1)] / df["X{}".format(j+1)]
/Users/xinruoli/Desktop/Interpretation-DNN-main/create_dataset.py:76: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ratios_df[column] = df["X{}".format(i+1)] / df["X{}".format(j+1)]
/Users/xinruoli/Desktop/Interpretation-DNN-main/create_dataset.py:76: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.in

### Linear Regression for Financial Ratios

In [11]:
from sklearn.metrics import mean_squared_error, precision_score, recall_score, f1_score, roc_auc_score

# ... (Previous code remains unchanged)
# Define features and target variable in ratios_df
xcols = [col for col in ratios_df.columns if col != 'status_label']
X = ratios_df[xcols].values
y = ratios_df['status_label'].values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create linear regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model (for example, using mean squared error)
mse = mean_squared_error(y_test, predictions)
print('Mean Squared Error:', mse)
print('-------------------------------------------')
# Convert predictions to binary classes based on a threshold
threshold = 0.5  # Define a threshold (adjust according to your problem)
binary_predictions = (predictions >= threshold).astype(int)

# Ensure y_test is binary as well (convert it to binary if it's not)
y_test_binary = (y_test >= threshold).astype(int)

# Evaluate binary predictions using classification metrics
precision = precision_score(y_test_binary, binary_predictions)
recall = recall_score(y_test_binary, binary_predictions)


print('Precision:', precision)
print('Recall:', recall)
print('-------------------------------------------')
coef_df = pd.DataFrame({'Feature': xcols, 'Coefficient': model.coef_})


# using abs value, descending order
sorted_coef_df = coef_df.reindex(coef_df['Coefficient'].abs().sort_values(ascending=False).index)


print('Coefficients sorted by absolute value in descending order:')
for index, row in sorted_coef_df.iterrows():
    print(row['Feature'], ':', row['Coefficient'])



Mean Squared Error: 5.0663839146170035
-------------------------------------------
Precision: 0.679980153807988
Recall: 0.6127878381399509
-------------------------------------------
Coefficients sorted by absolute value in descending order:
X7/X9 : -12123126377.387352
X7/X16 : 12123126377.387331
X9/X17 : -677388293.6033937
X16/X17 : 677384807.5141642
X2/X16 : 324866178.9838262
X2/X9 : -324865939.7808868
X5/X9 : -306599012.49855417
X5/X16 : 306599012.49847066
X1/X9 : 289809589.0248976
X1/X16 : -289809589.02486986
X4/X9 : -81328592.12175809
X4/X16 : 81326981.13704933
X16/X18 : 62004838.4949423
X9/X18 : -61529595.19595797
X3/X9 : 58427589.24919671
X3/X16 : -58427531.79922068
X8/X16 : 41289243.87248994
X8/X9 : -41289243.87232526
X9/X16 : -8197960.3030027775
X6/X9 : -2160844.493140321
X6/X16 : 2160844.4931186587
X4/X18 : -540345.0689368919
X9/X15 : -120266.05646818131
X2/X15 : 103803.66126994217
X9/X14 : 57310.89810799435
X12/X18 : 51194.27706400305
X2/X14 : -49384.16705469766
X12/X13 : -2

### Logisic Regression for Original Values(X1-X18)

In [9]:
DATAPATH = r'american_bankruptcy.csv'
data = load_data(DATAPATH)
xcols = [col for col in data.columns if col != 'status_label']
X = data[xcols].values
y = data['status_label'].values
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create logistic regression model
model = LogisticRegression(max_iter=1000)

# Train the model
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)

beta = dict( zip( xcols, list(model.coef_[0]) )) 

# using abs value, descending order
sorted_beta = sorted(beta.items(), key=lambda x: abs(x[1]), reverse=True)

print('Accuracy:', accuracy)
print('Coefficients sorted by absolute value in descending order:')
for item in sorted_beta:
    print(item[0], ":", item[1])

100%|██████████| 8971/8971 [01:11<00:00, 125.82it/s]


Accuracy: 0.6504119349810733
Coefficients sorted by absolute value in descending order:
X1 : 0.000773763373760542
X8 : 0.000756509329110712
X18 : -0.0007484906640521533
X7 : 0.000702792360616653
X6 : 0.000668015171216988
X17 : -0.000634898798221866
X5 : -0.000600414224945115
X2 : 0.000596205069686775
X12 : 0.00035034940213816095
X3 : -0.00027233725201490156
X14 : -0.000268142564660166
X13 : 0.0002047198024047852
X11 : 0.00011603622756377148
X4 : 7.801215013077694e-05
X9 : 6.52231791079112e-05
X16 : 6.52231791079112e-05
X15 : 4.2878053971405306e-05
X10 : 1.809492787961733e-05
